In [14]:
from dotenv import load_dotenv
import os
load_dotenv()
config = {
    "GOOGLE_API_KEY": os.getenv("GOOGLE_API_KEY")
}
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [15]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'